| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| C | array (Ny, Nx, 2, 2) | Cauchy-Green strain tensor over meshgrid |
| tensorline | string | Specifies type of tensorline: {shrinklines, stretchlines} |
| min_distance | float | minimum distance between local maxima |
| max_length | float | y-coordinate of vortex center |
| step_size | float | step size used for integration of shearline |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/Src")

In [ ]:
# Import numpy
import numpy as np

# Import function which calculates eigenvalues/eigenvectors in 2D
from ipynb.fs.defs.loc_max import _loc_max

# Import function which re-orients the vector field
from ipynb.fs.defs.orient_vectorfield import _orient_vectorfield

# Import function which does one step of the RK45 integration (designed for tensorlines)
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines

# Import function which calculates eigenvalues/eigenvectors in 2D
from ipynb.fs.defs.eigen import eigen_C

# Import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [2]:
def _tensorlines(X, Y, C, tensorline, min_distance, max_length, step_size):

    lamda = np.zeros((C.shape[0], C.shape[1]))*np.nan
    eigenvector_field = np.zeros((C.shape[0], C.shape[1], 2))
    
    for i in range(lamda.shape[0]):
        for j in range(lamda.shape[1]):
            lambda_1[i, j], lambda_2[i, j], v1[i,j,:], _ = eigen_C(C[i, j, :, :])
    
    if tensorline == "stretchline":
        
        eigenvector_field = np.zeros((C.shape[0], C.shape[1], 2))
    
    # Find local extrema of lambda_2-field
    peak_x, peak_y, peak_field = _loc_max(max_distance, X, Y, lambda_2)[2:]
    
    # vector_field
    vector_field = v1
    
    # Defined domain
    defined_domain = np.isfinite(lambda_2).astype(int)
    
    # Set values of lambda to 0 for interpolation
    for i in range(lambda_1.shape[0]):
        for j in range(lambda_1.shape[1]):
            if np.isnan(lambda_1[i, j]):
                lambda_1[i, j] = 0
                lambda_2[i, j] = 0
        
    # Interpolate Eigenvalue field
    #interp_lambda_1 = RBS(Y[:,0], X[0,:], lambda_1, kx = 1, ky = 1)
    interp_lambda = RBS(Y[:,0], X[0,:], lambda_2, kx = 1, ky = 1)
    
    # Define list of shrinklines and associated lambda value
    shrinklines = [[], []]
    
    # Iterate over all local maxima
    for i in range(len(peak_x)):
        
        shrinklines_forw = [[], []]
        shrinklines_back = [[], []]
        
        # Local maxima point
        x = np.array([peak_x[i], peak_y[i]])
        
        # Boolean forward Iteration
        bool_forward, bool_backward = True, True
        
        # Check if local maximum is close to shrinkline
        for j in range(len(shrinklines[0])):
                
            for k in range(len(shrinklines[0][j])):
                    
                if np.sqrt((x[0]-shrinklines[0][j][k])**2+(x[1]-shrinklines[1][j][k])**2) < max_distance:
                        bool_forward = False
                        bool_backward = False
                        break
        
        # Start integration only if local maxima is not close than 'max_distance' to shrinkline
        if bool_forward and bool_backward:
            
            # Starting point of integration
            x_forward = x
            x_backward = x
            
            # Append starting point to list containing positions of forward shrinklines
            for ii in range(2):
                shrinklines_forw[ii].append(x[ii])
                
            # Check orientation of vector-field and rieorient if needed.
            vx, vy = _orient_vectorfield(X, Y, x, vector_field)
            
            # Initial vector orientation
            x_prime_forward = np.array([vx, vy])
            x_prime_backward = -np.array([vx, vy])
            
            # Initial distance
            dist = 0
            
            # Start Integration with dummy variable 's_array'
            while bool_forward or bool_backward:
                
                # Integrate only if 'x_prime_forward' is defined and 'bool_forward == True'
                if bool_forward and x_prime_forward is not None:
                    
                    # RK4 integration for tensorline
                    x_forward, x_prime_forward = _RK4_tensorlines(X, Y, defined_domain, x_forward, x_prime_forward, step_size, vector_field, interp_lambda) 
                        
                    if x_forward is not None:
                        
                        if np.sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2) < 10**(-6):
                            bool_forward = False
                        
                        # Compute length of shrinkline
                        dist += np.sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2)*step_size
                        
                        # If distance is below length of shrinkline --> append point to shrinkline
                        if dist < max_length:
                            for ii in range(2):
                                shrinklines_forw[ii].append(x_forward[ii])
                    
                    else:
                        
                        bool_forward = False
                
                # Integrate only if 'x_prime_backward' is defined and 'bool_backward == True'
                if bool_backward and x_prime_backward is not None:
                    
                    # RK4 integration for tensorline
                    x_backward, x_prime_backward = _RK4_tensorlines(X, Y, defined_domain, x_backward, x_prime_backward, step_size, vector_field, interp_lambda) 
                    
                    if x_backward is not None:
                        
                        if np.sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2) < 10**(-6):
                            bool_backward = False
                        
                        # Compute shrinkline
                        dist += np.sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2)*step_size
                        
                        # If distance is below length of shrinkline --> append point to shrinkline
                        if dist < max_length:
                            for ii in range(2):
                                shrinklines_back[ii].append(x_backward[ii])
                    
                    else:
                        
                        bool_backward = False
                        
            # Append backward and forward shrinkline
            for ii in range(2):
                shrinklines[ii].append(np.append(np.flip(shrinklines_back[ii]), shrinklines_forw[ii]))
    
    return shrinklines